# Spark Structured Streaming 

`Structured Streaming` — это масштабируемая и отказоустойчивая библиотека для потоковой обработки, построенный на базе `Spark SQL`. Основная идея - с потоковыми вычислениями можно работать так же, как и со статическими данными. 

In [ ]:
import os
import time

import dbldatagen as dg
import pyspark.sql.functions as F
import numpy as np
import pandas as pd
from pyspark.sql import SparkSession
from pyspark.sql.functions import window, session_window
from pyspark.sql.types import StringType

 # Если переменная окружения  `JAVA_HOME` не установлена, то тут можно её указать.
os.environ["JAVA_HOME"] = "..."

Создаем сессию `Spark`, как обычно

In [2]:
spark = SparkSession \
    .builder \
    .appName("structured") \
    .config("spark.sql.streaming.forceDeleteTempCheckpointLocation", True) \
    .getOrCreate()

spark.sparkContext.setLogLevel("ERROR")

Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
25/11/19 14:33:15 WARN Utils: Your hostname, burg, resolves to a loopback address: 127.0.1.1; using 10.0.2.15 instead (on interface enp0s3)
25/11/19 14:33:15 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/11/19 14:33:23 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Проверяем, создадим "статический" DataFrame

In [3]:
df = spark.createDataFrame([("row1", 10), ("row2", 200)], ["column1", "columns2"])

Модель исполнения:
1. Входные данные поступают пачками (`mini-batch`) и добавляются к некоторому "бесконечному" DataFrame. Размер и частота появления `mini-batch` зависит от источника (генерируются "по триггеру").
2. Пользователем описываются некоторые операции по преобразованию "бесконечного DataFrame", как в "статическом" Spark.
3. В итоге получается "результирующий DataFrame", который является результатом работы и записывается во внешний источник (топик Kafka, консоль, файлы, etc)

Создаем Streaming DataFrame, описывая процесс получения данных из какого-нибудь источника. Поддерживается 4 встроенных источника:
- Kafka (`kafka`)
- Файлы 
- Сеть (`socket`)
- Генерация DataFrame вида `(timestamp TIMESTAMP, value LONG )`, для тестовых целей (`rate`)

In [4]:
# Для Kafka нужно указать топик

# df = spark \
#   .readStream \
#   .format("kafka") \
#   .option("kafka.bootstrap.servers", "localhost:9092") \
#   .option("subscribePattern", "topic*") \
#   .option("startingOffsets", "earliest") \
#   .load()


# Будет создаваться 10 записей в секунду 
df = spark \
    .readStream \
    .format("rate") \
    .option("rowsPerSecond", "10") \
    .load()

df.printSchema()

root
 |-- timestamp: timestamp (nullable = true)
 |-- value: long (nullable = true)



Можно запустить процесс обработки, дать поработать 10 секунд и остановить. Данные накапливаются в течение некоторого времени по триггеру в так называемый `mini-batch` и обрабатываются. Затем обновления добавляются в "бесконечный" DataFrame. Режим вывода может быть:
- `update` - выводить только обновленные строки
- `complete` - DataFrame полностью
- `append` - новые строки

Не все эти режимы доступны, зависит от применяемых операций обработки DataFrame. Результат будет выводиться в консоль. 

In [14]:
query = df \
    .writeStream \
    .outputMode("append") \
    .format("console") \
    .option("truncate", "false") \
    .start()

time.sleep(10)

query.stop()

-------------------------------------------
Batch: 0
-------------------------------------------
+---------+-----+
|timestamp|value|
+---------+-----+
+---------+-----+

-------------------------------------------
Batch: 1
-------------------------------------------
+-----------------------+-----+
|timestamp              |value|
+-----------------------+-----+
|2025-11-12 12:12:12.111|0    |
|2025-11-12 12:12:12.511|4    |
|2025-11-12 12:12:12.911|8    |
|2025-11-12 12:12:13.311|12   |
|2025-11-12 12:12:13.711|16   |
|2025-11-12 12:12:14.111|20   |
|2025-11-12 12:12:14.511|24   |
|2025-11-12 12:12:14.911|28   |
|2025-11-12 12:12:15.311|32   |
|2025-11-12 12:12:15.711|36   |
|2025-11-12 12:12:16.111|40   |
|2025-11-12 12:12:16.511|44   |
|2025-11-12 12:12:16.911|48   |
|2025-11-12 12:12:17.311|52   |
|2025-11-12 12:12:17.711|56   |
|2025-11-12 12:12:18.111|60   |
|2025-11-12 12:12:18.511|64   |
|2025-11-12 12:12:18.911|68   |
|2025-11-12 12:12:19.311|72   |
|2025-11-12 12:12:19.711|76  

Библиотека [dbldatagen](https://github.com/databrickslabs/dbldatagen) позволяет, для тестовых целей, генерировать DataFrame с заданной схемой и случайным содержимом. Создадим DataFrame с одной колонкой, в которой может быть одно из пяти заданных слов. 

In [5]:
# описываем данные, которые будут генерироваться
ds = dg.DataGenerator(spark, name="Words", rows=20, partitions=1) \
      .withColumn("word", StringType(), values=["hello", "world", "ok", "no", "yes"], weights=[1, 1, 2, 2, 2])

# создаем Streaming DataFrame
df = ds.build(withStreaming=True, options={'rowsPerSecond': 3})

df.printSchema()

root
 |-- word: string (nullable = false)



In [5]:

query = df \
    .writeStream \
    .outputMode("append") \
    .format("console") \
    .option("truncate", "false") \
    .start()

time.sleep(10)

query.stop()

-------------------------------------------
Batch: 0
-------------------------------------------
+----+
|word|
+----+
+----+

-------------------------------------------
Batch: 1
-------------------------------------------
+-----+
|word |
+-----+
|hello|
|world|
|ok   |
|ok   |
|no   |
|no   |
|yes  |
|yes  |
|hello|
+-----+

-------------------------------------------
Batch: 2
-------------------------------------------
+-----+
|word |
+-----+
|world|
|ok   |
|ok   |
|no   |
|no   |
|yes  |
+-----+

-------------------------------------------
Batch: 3
-------------------------------------------
+-----+
|word |
+-----+
|yes  |
|hello|
|world|
+-----+

-------------------------------------------
Batch: 4
-------------------------------------------
+----+
|word|
+----+
|ok  |
|ok  |
|no  |
+----+

-------------------------------------------
Batch: 5
-------------------------------------------
+----+
|word|
+----+
|no  |
|yes |
|yes |
+----+

-------------------------------------------
Ba

Теперь можно описать преобразования (подсчет слов) и выводить текущую статистику в консоль

In [ ]:

df = df.groupBy("word").count()

query = df \
    .writeStream \
    .outputMode("complete") \
    .format("console") \
    .option("truncate", "false") \
    .start()


time.sleep(140)

query.stop()

-------------------------------------------
Batch: 0
-------------------------------------------
+----+-----+
|word|count|
+----+-----+
+----+-----+



-------------------------------------------
Batch: 1
-------------------------------------------
+-----+-----+
|word |count|
+-----+-----+
|hello|12   |
|ok   |24   |
|no   |24   |
|world|12   |
|yes  |24   |
+-----+-----+



-------------------------------------------
Batch: 2
-------------------------------------------
+-----+-----+
|word |count|
+-----+-----+
|hello|24   |
|ok   |48   |
|no   |48   |
|world|24   |
|yes  |48   |
+-----+-----+



-------------------------------------------
Batch: 3
-------------------------------------------
+-----+-----+
|word |count|
+-----+-----+
|hello|35   |
|ok   |70   |
|no   |70   |
|world|35   |
|yes  |69   |
+-----+-----+



KeyboardInterrupt: 

-------------------------------------------
Batch: 4
-------------------------------------------
+-----+-----+
|word |count|
+-----+-----+
|hello|45   |
|ok   |88   |
|no   |88   |
|world|45   |
|yes  |88   |
+-----+-----+



-------------------------------------------
Batch: 5
-------------------------------------------
+-----+-----+
|word |count|
+-----+-----+
|hello|53   |
|ok   |106  |
|no   |104  |
|world|53   |
|yes  |104  |
+-----+-----+



-------------------------------------------
Batch: 6
-------------------------------------------
+-----+-----+
|word |count|
+-----+-----+
|hello|61   |
|ok   |121  |
|no   |120  |
|world|61   |
|yes  |120  |
+-----+-----+



-------------------------------------------
Batch: 7
-------------------------------------------
+-----+-----+
|word |count|
+-----+-----+
|hello|66   |
|ok   |130  |
|no   |130  |
|world|66   |
|yes  |130  |
+-----+-----+



-------------------------------------------
Batch: 8
-------------------------------------------
+-----+-----+
|word |count|
+-----+-----+
|hello|71   |
|ok   |140  |
|no   |140  |
|world|70   |
|yes  |140  |
+-----+-----+



Проэмулируем получение данных от трех IoT-устройств

In [6]:
ds = dg.DataGenerator(spark, name="IOT", rows=5000, partitions=1) \
      .withColumn("time", "timestamp", expr="now()") \
      .withColumn("sensor", StringType(), values=["sensor_1", "sensor_2", "sensor_3"]) \
      .withColumn("value", "integer", minValue=0, maxValue=10, random=True)


df = ds.build(withStreaming=True, options={'rowsPerSecond': 10})

df.printSchema()

root
 |-- time: timestamp (nullable = false)
 |-- sensor: string (nullable = false)
 |-- value: integer (nullable = true)



In [9]:
df = ds.build(withStreaming=True, options={'rowsPerSecond': 10})

df = df.groupBy("sensor").avg("value")

query = df \
    .writeStream \
    .outputMode("update") \
    .format("console") \
    .option("truncate", "false") \
    .start()

time.sleep(70)

query.stop()

-------------------------------------------
Batch: 0
-------------------------------------------
+------+----------+
|sensor|avg(value)|
+------+----------+
+------+----------+



-------------------------------------------
Batch: 1
-------------------------------------------
+--------+-----------------+
|sensor  |avg(value)       |
+--------+-----------------+
|sensor_1|4.380952380952381|
|sensor_2|5.445783132530121|
|sensor_3|5.180722891566265|
+--------+-----------------+



25/11/19 14:51:52 ERROR WriteToDataSourceV2Exec: Data source write support MicroBatchWrite[epoch: 2, writer: ConsoleWriter[numRows=20, truncate=false]] is aborting.
25/11/19 14:51:52 ERROR WriteToDataSourceV2Exec: Data source write support MicroBatchWrite[epoch: 2, writer: ConsoleWriter[numRows=20, truncate=false]] aborted.


Можно считать статистику по "окнам", которые образуются заданными временными интервалами

In [10]:
df = ds.build(withStreaming=True, options={'rowsPerSecond': 10})

windowed_df = df\
    .groupBy(
        window(df.time, "10 seconds"),
        df.sensor
    ).avg("value")


query = windowed_df \
    .writeStream \
    .outputMode("update") \
    .format("console") \
    .option("truncate", "false") \
    .start()

time.sleep(70)

query.stop()

-------------------------------------------
Batch: 0
-------------------------------------------
+------+------+----------+
|window|sensor|avg(value)|
+------+------+----------+
+------+------+----------+



-------------------------------------------
Batch: 1
-------------------------------------------
+------------------------------------------+--------+------------------+
|window                                    |sensor  |avg(value)        |
+------------------------------------------+--------+------------------+
|{2025-11-19 14:54:20, 2025-11-19 14:54:30}|sensor_3|4.7631578947368425|
|{2025-11-19 14:54:20, 2025-11-19 14:54:30}|sensor_1|5.077922077922078 |
|{2025-11-19 14:54:20, 2025-11-19 14:54:30}|sensor_2|4.1558441558441555|
+------------------------------------------+--------+------------------+



-------------------------------------------
Batch: 2
-------------------------------------------
+------------------------------------------+--------+-----------------+
|window                                    |sensor  |avg(value)       |
+------------------------------------------+--------+-----------------+
|{2025-11-19 14:54:40, 2025-11-19 14:54:50}|sensor_1|4.940298507462686|
|{2025-11-19 14:54:40, 2025-11-19 14:54:50}|sensor_3|5.537313432835821|
|{2025-11-19 14:54:40, 2025-11-19 14:54:50}|sensor_2|5.106060606060606|
+------------------------------------------+--------+-----------------+



25/11/19 14:55:16 ERROR WriteToDataSourceV2Exec: Data source write support MicroBatchWrite[epoch: 3, writer: ConsoleWriter[numRows=20, truncate=false]] is aborting.
25/11/19 14:55:16 ERROR WriteToDataSourceV2Exec: Data source write support MicroBatchWrite[epoch: 3, writer: ConsoleWriter[numRows=20, truncate=false]] aborted.
25/11/19 14:55:16 ERROR Utils: Aborting task
org.apache.spark.SparkException: [CANNOT_WRITE_STATE_STORE.CANNOT_COMMIT] Error writing state store files for provider HDFSStateStore[id=(op=0,part=26),dir=file:/tmp/temporary-34bf4bd1-09de-41c4-bcec-c2624c6c72ed/state/0/26]. Cannot perform commit during state checkpoint. SQLSTATE: 58030
	at org.apache.spark.sql.errors.QueryExecutionErrors$.failedToCommitStateFileError(QueryExecutionErrors.scala:2206)
	at org.apache.spark.sql.execution.streaming.state.HDFSBackedStateStoreProvider$HDFSBackedStateStore.commit(HDFSBackedStateStoreProvider.scala:182)
	at org.apache.spark.sql.execution.streaming.state.StreamingAggregationState

Окна могут быть накладываться друг на друга или быть привязаны к какому-то полю события ("сессионные")
.
![](https://spark.apache.org/docs/latest/img/structured-streaming-time-window-types.jpg)

In [ ]:
# Пересекающиеся - 10 секунд каждые 5.
windowed_df = df\
   .groupBy(
        window(df.time, "10 seconds", "5 seconds"),
        df.sensor) \
    .avg("value")

# Разный размер окна в зависимости от датчика
sw = session_window(df.time, \
    F.when(df.sensor == "sensor_3", "5 seconds") \
     .when(df.sensor == "sensor_2", "10 seconds") \
     .otherwise("5 seconds"))

windowed_df = df\
    .groupBy(
        sw,
        df.sensor) \
    .avg("value")

Событие может быть создано намного раньше времени физической обработки Spark'ом. Например, это может быть связано с высокой нагрузкой или проблемой с сетью. Обработка такого события приведет к изменению исторических данных. Для того, чтобы избежать подобного, можно воспользоваться "watermark", указав максимальное время между значением поля времени и временем обработки события. 

In [ ]:

windowed_df = df\
    .withWatermark("time", "15 seconds") \
    .groupBy(
        window(df.time, "10 seconds"),
        df.sensor
    ).avg("value")

`Spark Streaming` поддерживает операции вида `join` между двумя датафреймами. Причем датафрейм может быть статическим или динамическим. 

In [19]:
df = ds.build(withStreaming=True, options={'rowsPerSecond': 10})

descr_df = spark.createDataFrame([("sensor_1", "Sensor #1"), ("sensor_2", "Sensor #2"), ("sensor_3", "Sensor #3")], ["sensor", "description"])

res_df = df.join(descr_df, "sensor")

query = res_df \
    .writeStream \
    .outputMode("append") \
    .format("console") \
    .option("truncate", "false") \
    .start()

time.sleep(30)

query.stop()

-------------------------------------------
Batch: 0
-------------------------------------------
+------+----+-----+-----------+
|sensor|time|value|description|
+------+----+-----+-----------+
+------+----+-----+-----------+



-------------------------------------------
Batch: 1
-------------------------------------------
+--------+--------------------------+-----+-----------+
|sensor  |time                      |value|description|
+--------+--------------------------+-----+-----------+
|sensor_1|2024-10-10 10:27:09.879887|10   |Sensor #1  |
|sensor_1|2024-10-10 10:27:09.879887|2    |Sensor #1  |
|sensor_1|2024-10-10 10:27:09.879887|0    |Sensor #1  |
|sensor_1|2024-10-10 10:27:09.879887|5    |Sensor #1  |
|sensor_1|2024-10-10 10:27:09.879887|9    |Sensor #1  |
|sensor_1|2024-10-10 10:27:09.879887|8    |Sensor #1  |
|sensor_1|2024-10-10 10:27:09.879887|5    |Sensor #1  |
|sensor_1|2024-10-10 10:27:09.879887|8    |Sensor #1  |
|sensor_1|2024-10-10 10:27:09.879887|8    |Sensor #1  |
|sensor_1|2024-10-10 10:27:09.879887|8    |Sensor #1  |
|sensor_1|2024-10-10 10:27:09.879887|6    |Sensor #1  |
|sensor_1|2024-10-10 10:27:09.879887|4    |Sensor #1  |
|sensor_1|2024-10-10 10:27:09.879887|4    |Sensor #1  |
|sensor

-------------------------------------------
Batch: 2
-------------------------------------------
+--------+--------------------------+-----+-----------+
|sensor  |time                      |value|description|
+--------+--------------------------+-----+-----------+
|sensor_1|2024-10-10 10:27:12.599167|7    |Sensor #1  |
|sensor_1|2024-10-10 10:27:12.599167|4    |Sensor #1  |
|sensor_1|2024-10-10 10:27:12.599167|3    |Sensor #1  |
|sensor_1|2024-10-10 10:27:12.599167|9    |Sensor #1  |
|sensor_1|2024-10-10 10:27:12.599167|8    |Sensor #1  |
|sensor_1|2024-10-10 10:27:12.599167|5    |Sensor #1  |
|sensor_1|2024-10-10 10:27:12.599167|7    |Sensor #1  |
|sensor_1|2024-10-10 10:27:12.599167|9    |Sensor #1  |
|sensor_1|2024-10-10 10:27:12.599167|7    |Sensor #1  |
|sensor_1|2024-10-10 10:27:12.599167|8    |Sensor #1  |
|sensor_2|2024-10-10 10:27:12.599167|5    |Sensor #2  |
|sensor_2|2024-10-10 10:27:12.599167|3    |Sensor #2  |
|sensor_2|2024-10-10 10:27:12.599167|8    |Sensor #2  |
|sensor

-------------------------------------------
Batch: 3
-------------------------------------------
+--------+--------------------------+-----+-----------+
|sensor  |time                      |value|description|
+--------+--------------------------+-----+-----------+
|sensor_1|2024-10-10 10:27:15.726165|3    |Sensor #1  |
|sensor_1|2024-10-10 10:27:15.726165|9    |Sensor #1  |
|sensor_1|2024-10-10 10:27:15.726165|10   |Sensor #1  |
|sensor_1|2024-10-10 10:27:15.726165|9    |Sensor #1  |
|sensor_1|2024-10-10 10:27:15.726165|1    |Sensor #1  |
|sensor_1|2024-10-10 10:27:15.726165|7    |Sensor #1  |
|sensor_1|2024-10-10 10:27:15.726165|7    |Sensor #1  |
|sensor_1|2024-10-10 10:27:15.726165|1    |Sensor #1  |
|sensor_1|2024-10-10 10:27:15.726165|6    |Sensor #1  |
|sensor_1|2024-10-10 10:27:15.726165|9    |Sensor #1  |
|sensor_2|2024-10-10 10:27:15.726165|7    |Sensor #2  |
|sensor_2|2024-10-10 10:27:15.726165|10   |Sensor #2  |
|sensor_2|2024-10-10 10:27:15.726165|2    |Sensor #2  |
|sensor

-------------------------------------------
Batch: 4
-------------------------------------------
+--------+--------------------------+-----+-----------+
|sensor  |time                      |value|description|
+--------+--------------------------+-----+-----------+
|sensor_1|2024-10-10 10:27:17.939313|8    |Sensor #1  |
|sensor_1|2024-10-10 10:27:17.939313|6    |Sensor #1  |
|sensor_1|2024-10-10 10:27:17.939313|0    |Sensor #1  |
|sensor_1|2024-10-10 10:27:17.939313|6    |Sensor #1  |
|sensor_1|2024-10-10 10:27:17.939313|0    |Sensor #1  |
|sensor_1|2024-10-10 10:27:17.939313|9    |Sensor #1  |
|sensor_2|2024-10-10 10:27:17.939313|4    |Sensor #2  |
|sensor_2|2024-10-10 10:27:17.939313|4    |Sensor #2  |
|sensor_2|2024-10-10 10:27:17.939313|7    |Sensor #2  |
|sensor_2|2024-10-10 10:27:17.939313|0    |Sensor #2  |
|sensor_2|2024-10-10 10:27:17.939313|4    |Sensor #2  |
|sensor_2|2024-10-10 10:27:17.939313|0    |Sensor #2  |
|sensor_2|2024-10-10 10:27:17.939313|10   |Sensor #2  |
|sensor

...

-------------------------------------------
Batch: 6
-------------------------------------------
+--------+--------------------------+-----+-----------+
|sensor  |time                      |value|description|
+--------+--------------------------+-----+-----------+
|sensor_1|2024-10-10 10:27:22.601582|6    |Sensor #1  |
|sensor_1|2024-10-10 10:27:22.601582|9    |Sensor #1  |
|sensor_1|2024-10-10 10:27:22.601582|7    |Sensor #1  |
|sensor_1|2024-10-10 10:27:22.601582|6    |Sensor #1  |
|sensor_1|2024-10-10 10:27:22.601582|8    |Sensor #1  |
|sensor_1|2024-10-10 10:27:22.601582|10   |Sensor #1  |
|sensor_1|2024-10-10 10:27:22.601582|1    |Sensor #1  |
|sensor_2|2024-10-10 10:27:22.601582|7    |Sensor #2  |
|sensor_2|2024-10-10 10:27:22.601582|2    |Sensor #2  |
|sensor_2|2024-10-10 10:27:22.601582|6    |Sensor #2  |
|sensor_2|2024-10-10 10:27:22.601582|0    |Sensor #2  |
|sensor_2|2024-10-10 10:27:22.601582|7    |Sensor #2  |
|sensor_2|2024-10-10 10:27:22.601582|2    |Sensor #2  |
|sensor

-------------------------------------------
Batch: 7
-------------------------------------------
+--------+--------------------------+-----+-----------+
|sensor  |time                      |value|description|
+--------+--------------------------+-----+-----------+
|sensor_1|2024-10-10 10:27:24.743627|6    |Sensor #1  |
|sensor_1|2024-10-10 10:27:24.743627|10   |Sensor #1  |
|sensor_1|2024-10-10 10:27:24.743627|9    |Sensor #1  |
|sensor_1|2024-10-10 10:27:24.743627|7    |Sensor #1  |
|sensor_1|2024-10-10 10:27:24.743627|0    |Sensor #1  |
|sensor_1|2024-10-10 10:27:24.743627|9    |Sensor #1  |
|sensor_1|2024-10-10 10:27:24.743627|10   |Sensor #1  |
|sensor_2|2024-10-10 10:27:24.743627|4    |Sensor #2  |
|sensor_2|2024-10-10 10:27:24.743627|6    |Sensor #2  |
|sensor_2|2024-10-10 10:27:24.743627|7    |Sensor #2  |
|sensor_2|2024-10-10 10:27:24.743627|5    |Sensor #2  |
|sensor_2|2024-10-10 10:27:24.743627|4    |Sensor #2  |
|sensor_2|2024-10-10 10:27:24.743627|3    |Sensor #2  |
|sensor

-------------------------------------------
Batch: 8
-------------------------------------------
+--------+--------------------------+-----+-----------+
|sensor  |time                      |value|description|
+--------+--------------------------+-----+-----------+
|sensor_1|2024-10-10 10:27:28.375509|10   |Sensor #1  |
|sensor_1|2024-10-10 10:27:28.375509|9    |Sensor #1  |
|sensor_1|2024-10-10 10:27:28.375509|2    |Sensor #1  |
|sensor_1|2024-10-10 10:27:28.375509|2    |Sensor #1  |
|sensor_1|2024-10-10 10:27:28.375509|1    |Sensor #1  |
|sensor_1|2024-10-10 10:27:28.375509|9    |Sensor #1  |
|sensor_1|2024-10-10 10:27:28.375509|6    |Sensor #1  |
|sensor_1|2024-10-10 10:27:28.375509|1    |Sensor #1  |
|sensor_1|2024-10-10 10:27:28.375509|1    |Sensor #1  |
|sensor_1|2024-10-10 10:27:28.375509|4    |Sensor #1  |
|sensor_1|2024-10-10 10:27:28.375509|10   |Sensor #1  |
|sensor_1|2024-10-10 10:27:28.375509|1    |Sensor #1  |
|sensor_1|2024-10-10 10:27:28.375509|0    |Sensor #1  |
|sensor

-------------------------------------------
Batch: 9
-------------------------------------------
+--------+--------------------------+-----+-----------+
|sensor  |time                      |value|description|
+--------+--------------------------+-----+-----------+
|sensor_1|2024-10-10 10:27:31.864305|7    |Sensor #1  |
|sensor_1|2024-10-10 10:27:31.864305|7    |Sensor #1  |
|sensor_1|2024-10-10 10:27:31.864305|6    |Sensor #1  |
|sensor_1|2024-10-10 10:27:31.864305|6    |Sensor #1  |
|sensor_1|2024-10-10 10:27:31.864305|9    |Sensor #1  |
|sensor_1|2024-10-10 10:27:31.864305|5    |Sensor #1  |
|sensor_1|2024-10-10 10:27:31.864305|8    |Sensor #1  |
|sensor_1|2024-10-10 10:27:31.864305|10   |Sensor #1  |
|sensor_1|2024-10-10 10:27:31.864305|9    |Sensor #1  |
|sensor_1|2024-10-10 10:27:31.864305|2    |Sensor #1  |
|sensor_2|2024-10-10 10:27:31.864305|9    |Sensor #2  |
|sensor_2|2024-10-10 10:27:31.864305|8    |Sensor #2  |
|sensor_2|2024-10-10 10:27:31.864305|1    |Sensor #2  |
|sensor

24/10/10 10:27:35 ERROR WriteToDataSourceV2Exec: Data source write support MicroBatchWrite[epoch: 10, writer: ConsoleWriter[numRows=20, truncate=false]] is aborting.
24/10/10 10:27:35 ERROR WriteToDataSourceV2Exec: Data source write support MicroBatchWrite[epoch: 10, writer: ConsoleWriter[numRows=20, truncate=false]] aborted.


In [ ]:
df = ds.build(withStreaming=True, options={'rowsPerSecond': 10})

windows_df_5 = df \
    .withWatermark("time", "30 seconds") \
    .groupBy(
        window("time", "5 seconds"),
        "sensor") \
    .agg(F.avg("value").alias("value")) \
    .select(F.window_time("window").alias("time"), "value", "sensor")


windows_df_10 = df \
    .withWatermark("time", "20 seconds") \
    .groupBy(
        window("time", "10 seconds"),
        "sensor") \
    .agg(F.avg("value").alias("value")) \
    .select(F.window_time("window").alias("time"), "value", "sensor")    
 
 
res_df = windows_df_5.alias("df_5").join(
    windows_df_10.alias("df_10"),
    F.expr("""
        df_5.sensor = df_10.sensor  AND
        df_5.time >= df_10.time AND
        df_5.time <= df_10.time + interval 40 seconds
        """)
)

query = res_df \
    .writeStream \
    .outputMode("append") \
    .format("console") \
    .option("truncate", "false") \
    .start()

time.sleep(50)

query.stop()